In [ ]:
import tensorflow as tf
from tensorflow import keras as keras
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
from numpy.random import randn

In [ ]:
atom_pos1=np.load('/content/drive/MyDrive/DataSets/Atom_Positions_Switched_1.npy')
atom_pos2=np.load('/content/drive/MyDrive/DataSets/Atom_Positions_Switched_2.npy')
atom_pos3=np.load('/content/drive/MyDrive/DataSets/Atom_Positions_Switched_3.npy')

In [ ]:
atom_pos=np.concatenate((atom_pos1,atom_pos2,atom_pos3),axis=0)
np.shape(atom_pos)

In [ ]:
x_train=np.expand_dims(atom_pos[:6400],4)
x_test=np.expand_dims(atom_pos[6400:],4)

In [ ]:
import keras
from keras import layers
from keras.layers import LeakyReLU
import keras.backend as K
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=60, restore_best_weights=True)

input_img = keras.Input(shape=(64,64,64, 1))
x = layers.BatchNormalization()(input_img)
x = layers.Conv3D(16, 3, strides = (4,4,4),  padding='same')(input_img)
x = LeakyReLU(alpha=0.2)(x)
x = layers.Conv3D(8, 3, strides = (4,4,4),activation='tanh', padding='same')(x)
encoded = layers.Flatten()(x)

x = layers.Dense(512, activation=None)(encoded)
x = layers.Reshape((4,4,4,8))(x)
x = layers.Conv3DTranspose(4, 8, strides=(4,4,4), padding='same')(x)
x = LeakyReLU(alpha=0.2)(x)
x = layers.Conv3DTranspose(8, 8, strides=(4,4,4),  padding='same')(x)
x = LeakyReLU(alpha=0.2)(x)
decoded = layers.Conv3D(1, 3, activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss=K.binary_crossentropy, metrics=['mae'])
autoencoder.summary()

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=64,
                batch_size=32,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[callback])